# Visualize optimized models comparison results

This notebook scans the `optimized_models` directory for `comparison_*.json` files, aggregates the reported metrics (fps, avg_inference_time_ms, model_size_mb, etc.), and renders interactive Plotly charts to compare variants (original, fp16, int8, batch) across models. Run the code cell below to produce the charts.

In [10]:
# Scan optimized_models for comparison_*.json but only use the latest session per model — only direct folders named "seg_*" — aggregate and plot with Plotly
import json
from pathlib import Path
import pandas as pd
import plotly.express as px

# root should be the optimized_models folder (not the repo root)
root = Path("./")

rows = []
# iterate immediate child model folders whose name starts with 'seg_'
model_folders = sorted([p for p in root.iterdir() if p.is_dir() and p.name.startswith('seg_')])
for model_dir in model_folders:
    # find session subfolders inside the model_dir (e.g., 20250910_204734_000015)
    session_dirs = [d for d in model_dir.iterdir() if d.is_dir()]
    if not session_dirs:
        # sometimes files may be directly inside; check for comparison files directly
        candidates = list(model_dir.rglob('comparison_*.json'))
        if not candidates:
            continue
        latest_session_root = model_dir
    else:
        # choose the latest session by folder name (YYYYMMDD... lexicographical order works)
        latest_session = max(session_dirs, key=lambda d: d.name)
        latest_session_root = latest_session

    # search within the latest session root for comparison_*.json files
    found = list(latest_session_root.rglob('comparison_*.json'))
    if not found:
        # if nothing found, try one level deeper (some layouts have an extra model-named folder)
        deeper = []
        for child in latest_session_root.iterdir():
            if child.is_dir():
                deeper.extend(child.rglob('comparison_*.json'))
        found = deeper

    if not found:
        # nothing to process for this model
        print(f"No comparison_*.json for model folder {model_dir.name} (checked latest session {latest_session_root})")
        continue

    # process all comparison files found in the chosen latest session
    for p in found:
        try:
            data = json.loads(p.read_text(encoding='utf-8'))
        except Exception as e:
            print(f"Skipping {p}: {e}")
            continue
        model_label = model_dir.name
        # flatten the metrics
        for variant, metrics in data.items():
            row = dict(model=model_label, variant=variant, filepath=str(p), session=str(latest_session_root))
            if not isinstance(metrics, dict):
                # safety: sometimes JSON may be a list/other
                print(f"⚠️ Skipping {p} - invalid metrics format for variant {variant}")
                continue
            for k, v in metrics.items():
                row[k] = v
            rows.append(row)

if not rows:
    print('No comparison_*.json files found under', root, 'for latest sessions of seg_* folders')
else:
    df = pd.DataFrame(rows)
    # tidy up model names
    df['model'] = df['model'].astype(str)
    display(df.head())

    # Plot: FPS by model and variant
    if 'fps' in df.columns:
        fig1 = px.bar(df, x='model', y='fps', color='variant', barmode='group', title='FPS by model and variant')
        fig1.show()

    # Plot: inference time ms by model and variant
    if 'avg_inference_time_ms' in df.columns:
        fig2 = px.bar(df, x='model', y='avg_inference_time_ms', color='variant', barmode='group', title='Avg inference time (ms) by model and variant')
        fig2.update_yaxes(type='log')
        fig2.show()

    # Scatter: model size vs fps
    if 'model_size_mb' in df.columns and 'fps' in df.columns:
        fig3 = px.scatter(df, x='model_size_mb', y='fps', color='variant', hover_data=['model', 'filepath', 'session'], size='fps', title='Model size (MB) vs FPS')
        fig3.show()

    print('Done.')

⚠️ Skipping seg_b0_ade\20250910_211137_000015\seg_b0_ade\comparison_seg_b0_ade.json - invalid metrics format for variant checkpoint_filename
⚠️ Skipping seg_b0_city\20250910_211142_000016\seg_b0_city\comparison_seg_b0_city.json - invalid metrics format for variant checkpoint_filename
⚠️ Skipping seg_b1_ade\20250910_211148_000017\seg_b1_ade\comparison_seg_b1_ade.json - invalid metrics format for variant checkpoint_filename
⚠️ Skipping seg_b1_city\20250910_211154_000018\seg_b1_city\comparison_seg_b1_city.json - invalid metrics format for variant checkpoint_filename
⚠️ Skipping seg_b2_ade\20250910_211202_000019\seg_b2_ade\comparison_seg_b2_ade.json - invalid metrics format for variant checkpoint_filename
⚠️ Skipping seg_b2_city\20250910_211211_000020\seg_b2_city\comparison_seg_b2_city.json - invalid metrics format for variant checkpoint_filename
⚠️ Skipping seg_b3_ade\20250910_211222_000021\seg_b3_ade\comparison_seg_b3_ade.json - invalid metrics format for variant checkpoint_filename
⚠️ S

,model,variant,filepath,session,avg_inference_time_ms,fps,model_size_mb,batch_size
0,seg_b0_ade,original,seg_b0_ade\20250910_211137_000015\seg_b0_ade\c...,seg_b0_ade\20250910_211137_000015,83.865786,11.923814,14.329124,NaN
1,seg_b0_ade,fp16,seg_b0_ade\20250910_211137_000015\seg_b0_ade\c...,seg_b0_ade\20250910_211137_000015,45.358562,22.046554,7.164581,NaN
2,seg_b0_ade,int8,seg_b0_ade\20250910_211137_000015\seg_b0_ade\c...,seg_b0_ade\20250910_211137_000015,76.924610,12.999741,14.329124,NaN
3,seg_b0_ade,batch,seg_b0_ade\20250910_211137_000015\seg_b0_ade\c...,seg_b0_ade\20250910_211137_000015,73.013687,13.696062,14.329124,4.0
4,seg_b0_city,original,seg_b0_city\20250910_211142_000016\seg_b0_city...,seg_b0_city\20250910_211142_000016,84.230804,11.872141,14.200695,NaN


Done.
